In [1]:
# $ pip3 install requests
import requests

In [3]:
# $ pip3 install bs4
from bs4 import BeautifulSoup

In [4]:
# $ pip3 install python-forecastio
import forecastio

- api json을 사용 : 네이버 주식 데이터 크롤링
- api json을 사용, forecastio : dark sky api로 날씨 데이터를 수집
- bs4 : 네이버 실시간 키워드, 다음 실시간 키워드 크롤링
- web file url로 파일을 다운로드 받는 방법

In [61]:
from pandas.io.json import json_normalize

##### 네이버 주식데이터 가져오기

In [5]:
# url 만들기
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)
make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=10&page=1'

In [62]:
# 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
def get_data(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    return json_normalize(companys)    

#     df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"])
#     for company in companys:
#         df.loc[len(df)] = {
#             "종목":company["nm"],
#             "시세":company["nv"],
#             "전일비":company["cv"],
#             "등락율":company["cr"],
#             "시가총액":company["mks"],
#             "거래량":company["aq"],
#         }
#     return df

In [63]:
url = make_url(1500, 1)
df = get_data(url)
df.tail()

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
1453,79,8033,284990,-0.10,-10,22,0,KBSTAR 200에너지화학,9875,9885,5
1454,2,225,284980,-1.78,-175,21,0,KBSTAR 200금융,9680,9855,5
1455,0,50,295820,-0.80,-80,15,0,ARIRANG 200동일가중,9940,10020,5
1456,280,28489,295860,-1.11,-110,15,0,ARIRANG 200모멘텀,9770,9880,5
1457,278,28667,295890,-1.23,-120,14,0,ARIRANG 200로우볼,9665,9785,5


##### dark sky api
- 위도와 경도를 입력하면 날씨 정보를 보내주는 api
- https://api.darksky.net/

In [25]:
FORECAST_TOKEN = "c259d4aeb593bc5e83d0c3f7ed916f5d"

In [37]:
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}"\
.format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["timezone"], json_info["hourly"]["summary"]

In [38]:
lat = 37.512
lng = 126.954
forecast(lat, lng)

('Asia/Seoul', 'Mostly cloudy throughout the day.')

In [39]:
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHourly = forecast.hourly()
    return byHourly.summary

In [40]:
lat = 37.512
lng = 126.954
forecast2(lat, lng)

'Mostly cloudy throughout the day.'

##### bs4
- 네이버 키워드 랭킹 데이터 크롤링
- 다음 키워드 랭킹 데이터 크롤링

In [75]:
# dom.select - 여러개의 html 엘리먼트를 셀렉팅 할때 사용, 결과로 리스트 데이터를 리턴
# dom.select_one - 하나의 html 엘리먼트를 셀렉팅 할때 사용, 결과로 문자열 데이터를 리턴

def naver():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item")
    
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank": keyword.select_one(".ah_r").text,
            "keyword": keyword.select_one(".ah_k").text,
        }
        
    return df
    
naver_df = naver()    
naver_df["keyword"]

0          이집트 우루과이
1              뮤직뱅크
2              블랙핑크
3              광주날씨
4             포항 약국
5                쌈디
6            모로코 이란
7                살라
8     러시아 사우디 하이라이트
9               민갑룡
10        한국예술인복지재단
11              심영은
12              강혁민
13              ksl
14         포르투갈 스페인
15              박기량
16             경찰청장
17            국가장학금
18           프로듀스48
19            하트시그널
Name: keyword, dtype: object

In [53]:
import time
ls = []
for idx in range(3):
    print(idx)
    ls.append(naver())
    time.sleep(5)

0
1
2


In [64]:
# 다음의 실시간 이슈 검색어 10개 수집
# 네이버 20개 데이터와 다음 10개 데이터에서 중복되는 데이터를 출력

In [76]:
def daum():
    df = pd.DataFrame()
    response = requests.get("https://www.daum.net")
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords = dom.select(".realtime_part .list_hotissue.issue_row > li")
    rank = [ keyword.select_one(".ir_wa").text[:-1] for keyword in keywords ]
    keywords = [ keyword.select_one(".link_issue").text for keyword in keywords ]
    df["rank"] = rank
    df["keyword"] = keywords
    return df

daum_df = daum()
daum_df["keyword"]

0       민갑룡
1      경찰청장
2     국가장학금
3    한국장학재단
4       조현배
5      블랙핑크
6       심영은
7       안희정
8       김무성
9       이찬오
Name: keyword, dtype: object

In [83]:
# naver_df["keyword"], daum_df["keyword"]
result = [
    keyword 
    for keyword in naver_df["keyword"] 
    if daum_df["keyword"].str.contains(keyword).any()
]
result

['블랙핑크', '민갑룡', '심영은', '경찰청장', '국가장학금']

##### file down load : requests
- file의 url을 받아서 해당 url의 file을 다운로드 하는 방법

In [89]:
def download(title, download_link):
    response = requests.get(download_link, stream=True)
    file_size = 0
    with open(title, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): # byte
            if chunk:
                file_size += 1024
                f.write(chunk)
    return file_size

In [92]:
# url = "https://r1---sn-a5meknes.googlevideo.com/videoplayback?sparams=dur%2Cei%2Cid%2Cip%2Cipbits%2Citag%2Clmt%2Cmime%2Cmm%2Cmn%2Cms%2Cmv%2Cpl%2Cratebypass%2Crequiressl%2Csource%2Cexpire&requiressl=yes&mt=1529051193&expire=1529073303&itag=22&lmt=1529039497166480&ei=N3ojW4nHKMKz-AP295aoCw&fvip=4&ratebypass=yes&ip=23.80.157.92&pl=16&mime=video%2Fmp4&dur=34.342&mv=u&source=youtube&ms=au%2Crdu&c=WEB&mn=sn-a5meknes%2Csn-a5mlrnel&mm=31%2C29&ipbits=0&key=yt6&id=o-AI8QABbxxbajaC8XwEjgsjsDso7ZX19oANOhqxPDFYir&signature=91530523B732709286CB08EF9E999994BCB05EC3.1F3B0C7848839521DDC0CA4D1296183AAB0DC568&video_id=ShQZhdjqjsw&title=iPhone+X+%E2%80%94+How+to+shoot+using+backlight+%E2%80%94+Apple"
download_link = "https://bit.ly/2tbDDMN"
title = "video.mp4"
file_size = download(title, download_link)

In [94]:
str(round(file_size/1024/1024,2)) + "Mb"

'2.19Mb'